In [1]:
!ls

 orders.csv   restaurants.sql   sample_data  'users (1).json'


In [2]:
import pandas as pd
import sqlite3

In [3]:
orders = pd.read_csv('/content/orders.csv')
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
users = pd.read_json('/content/users (1).json')
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [5]:
users = pd.read_json('/content/users (1).json')
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
# SQLite in-memory database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Read SQL file
with open('/content/restaurants.sql', 'r') as file:
    sql_script = file.read()

# Execute SQL
cursor.executescript(sql_script)

# Load restaurants table
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
orders_users = pd.merge(
    orders,
    users,
    on='user_id',
    how='left'
)

final_df = pd.merge(
    orders_users,
    restaurants,
    on='restaurant_id',
    how='left'
)

final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df.to_csv('final_food_delivery_dataset.csv', index=False)

In [9]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [10]:
final_df = final_df.rename(columns={
    'restaurant_name_y': 'restaurant_name',
    'membership': 'membership_type'
})

final_df = final_df.drop(columns=['restaurant_name_x'])
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership_type,restaurant_name,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [11]:
from google.colab import files
files.download('final_food_delivery_dataset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
final_df[final_df['membership_type'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


In [13]:
final_df.groupby('cuisine')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


In [14]:
user_spend = final_df.groupby('user_id')['total_amount'].sum()
count_users = user_spend[user_spend > 1000].count()
count_users

np.int64(2544)

In [15]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = ['3.0–3.5', '3.6–4.0', '4.1–4.5', '4.6–5.0']

final_df['rating_range'] = pd.cut(final_df['rating'], bins=bins, labels=labels)

final_df.groupby('rating_range')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

/tmp/ipython-input-1348670951.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby('rating_range')['total_amount'] \


,total_amount
rating_range,
4.6–5.0,2197030.75
4.1–4.5,1960326.26
3.0–3.5,1881754.57
3.6–4.0,1717494.41


In [16]:
final_df[final_df['membership_type'] == 'Gold'] \
    .groupby('city')['total_amount'] \
    .mean() \
    .sort_values(ascending=False)

,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


In [17]:
restaurants.groupby('cuisine')['restaurant_id'] \
    .nunique() \
    .sort_values()

,restaurant_id
cuisine,
Chinese,120
Indian,126
Italian,126
Mexican,128


In [18]:
final_df.groupby('cuisine')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
cuisine,
Mexican,2085503.09
Italian,2024203.80
Indian,1971412.58
Chinese,1930504.65


In [19]:
gold_orders = final_df[final_df['membership_type'] == 'Gold'].shape[0]
total_orders = final_df.shape[0]

round((gold_orders / total_orders) * 100)

50

In [20]:
rest_stats = final_df.groupby('restaurant_name').agg(
    avg_order_value=('total_amount', 'mean'),
    total_orders=('order_id', 'count')
)

rest_stats[rest_stats['total_orders'] < 20] \
    .sort_values('avg_order_value', ascending=False)

,avg_order_value,total_orders
restaurant_name,,
Restaurant_294,1040.222308,13
Restaurant_262,1029.473333,18
Restaurant_77,1029.180833,12
Restaurant_193,1026.306667,15
Restaurant_7,1002.140625,16
...,...,...
Restaurant_184,621.828947,19
Restaurant_498,596.815556,18
Restaurant_192,589.972857,14


In [21]:
final_df.groupby(['membership_type', 'cuisine'])['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

membership_type  cuisine
Regular          Mexican    1072943.30
                 Italian    1018424.75
Gold             Mexican    1012559.79
                 Italian    1005779.05
Regular          Indian      992100.27
Gold             Indian      979312.31
                 Chinese     977713.74
Regular          Chinese     952790.91
Name: total_amount, dtype: float64

In [22]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'], dayfirst=True)
final_df['quarter'] = final_df['order_date'].dt.to_period('Q')

final_df.groupby('quarter')['total_amount'] \
    .sum() \
    .sort_values(ascending=False)

,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


In [23]:
final_df[final_df['membership_type'] == 'Gold'].shape[0]

4987

In [24]:
round(
    final_df[final_df['city'] == 'Hyderabad']['total_amount'].sum()
)

1889367

In [25]:
final_df['user_id'].nunique()

2883

In [26]:
round(
    final_df[final_df['membership_type'] == 'Gold']['total_amount'].mean(),
    2
)

np.float64(797.15)

In [27]:
final_df[final_df['rating'] >= 4.5].shape[0]

3374

In [28]:
# Step 1: Top revenue city among Gold members
top_gold_city = (
    final_df[final_df['membership_type'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

# Step 2: Orders count in that city (Gold only)
final_df[
    (final_df['membership_type'] == 'Gold') &
    (final_df['city'] == top_gold_city)
].shape[0]

1337

In [29]:
# Step 1: Top revenue city among Gold members
top_gold_city = (
    final_df[final_df['membership_type'] == 'Gold']
    .groupby('city')['total_amount']
    .sum()
    .idxmax()
)

# Step 2: Orders count in that city (Gold only)
final_df[
    (final_df['membership_type'] == 'Gold') &
    (final_df['city'] == top_gold_city)
].shape[0]

1337

In [30]:
pd.merge(orders,users,on="user-id",how="left")

KeyError: 'user-id'

In [31]:
pd.merge(orders, users, on="user_id", how="left")


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [32]:
restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

In [33]:
len(final_df)


10000

In [34]:
pd.merge(orders, users, on="user_id", how="left")

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,User_2528,Hyderabad,Gold
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,User_2867,Bangalore,Regular
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,User_522,Bangalore,Gold
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,User_319,Bangalore,Gold


In [35]:
orders + users  →  membership column final_df

SyntaxError: invalid character '→' (U+2192) (ipython-input-3799192040.py, line 1)

In [36]:
pd.merge(orders, restaurants, on="restaurant_id", how="left")

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,Restaurant_293,Italian,3.0
...,...,...,...,...,...,...,...,...,...
9995,9996,2528,249,21-05-2023,1211.96,Royal Kitchen North Indian,Restaurant_249,Italian,4.7
9996,9997,2867,267,06-08-2023,1188.05,Darbar Cafe Punjabi,Restaurant_267,Chinese,4.2
9997,9998,522,420,11-11-2023,979.44,Ruchi Tiffins Chinese,Restaurant_420,Italian,4.0
9998,9999,319,492,08-09-2023,1105.93,Swagath Kitchen North Indian,Restaurant_492,Italian,4.0
